# Implementing Simple RAG using ChromaDB and Langchain

Source [Medium](https://medium.com/@callumjmac/implementing-rag-in-langchain-with-chroma-a-step-by-step-guide-16fc21815339)

In [19]:
# Langchain dependencies
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate


import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

**Document loaders form [Langchain](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/)**

In [7]:
%ls 'data'

genAi_python_type_inference.pdf  pyty_paper.pdf*


## Load Documents

In [ ]:
# Directory to your pdf files:
DATA_PATH = "data"


def load_documents():
  """
    Load PDF documents from the specified directory using PyPDFDirectoryLoader.
    
    Returns:
    List of Document objects: Loaded PDF documents represented as Langchain Document objects.
  """
  
  # Initialize PDF loader with specified directory
  document_loader = PyPDFDirectoryLoader(DATA_PATH) 
  # Load PDF documents and return them as a list of Document objects
  return document_loader.load() 

documents = load_documents() # Call the function

# Inspect the contents of the first document as well as metadata
# print(documents[0])

## Chunking

In RAG systems, “chunking” refers to the segmentation of input text into shorter and more meaningful units. This enables the system to efficiently pinpoint and retrieve relevant pieces of information. The quality of chunks are **essential** to how effective your system will be.

The most **important** thing to consider when **deciding a chunking strategy** is the structure of the documents that you are loading into your vector database. If the documents contain similar-length paragraphs, it would be useful to consider this when determining the size of the chunk.

----

If your `chunk_size` is too large, then you will be inputing a lot of noisy and unwanted context to the final LLM query. Further, as LLMs are limited by their context window size, the larger the chunk, the fewer pieces of relevant information you can include as context to your query.

The `chunk_overlap` refers to intentionally duplicating tokens at the beginning and end of each adjacent chunk. This helps retain additional context from neighbouring chunks which can improve the quality of the information passed into the prompt.

When deploying these systems to real-world applications, it is important to plot distributions of text lengths in your data, and tune these parameters based on experimentation of parameters such as `chunk_size` and `chunk_overlap`.

In [10]:
def split_text(documents: list[Document]):
  """
  Split the text content of the given list of Document objects into smaller chunks.
  
  Args:
    documents (list[Document]): List of Document objects containing text content to split.
  Returns:
    list[Document]: List of Document objects representing the split text chunks.
  """
  
  # Initialize text splitter with specified parameters
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Size of each chunk in characters
    chunk_overlap=100, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
  )

  # Split documents into smaller chunks using text splitter
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

  # Print example of page content and metadata for a chunk
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks # Return the list of split text chunks

## Create VectorDB (Chroma)

It's encourageg experimentation with open-sourced models such as **Llama3** (try the 8B parameter version first, especially if your system needs to be 100% local). If you are working on a very niche or nuanced use case, off-the-shelf embedding models may not be useful. Therefore, you might want to investigate fine-tuning the embedding model on the domain data to improve the retrieval quality.



In [13]:
# Path to the directory to save Chroma database
CHROMA_PATH = "chromaDB"
def save_to_chroma(chunks: list[Document]):
  """
  Save the given list of Document objects to a Chroma database.
  
  Args:
  chunks (list[Document]): List of Document objects representing text chunks to save.
  Returns:
  None
  """
  
  load_dotenv()
  GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using GEMINI embeddings
  db = Chroma.from_documents(
    chunks,
    GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY),
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [14]:
def generate_data_store():
  """
  Function to generate vector database in chroma from documents.
  """
  documents = load_documents() # Load documents from a source
  chunks = split_text(documents) # Split documents into manageable chunks
  save_to_chroma(chunks) # Save the processed data to a data store


# Load environment variables from a .env file
load_dotenv()
# Generate the data store
generate_data_store()


Split 25 documents into 733 chunks.
Generative Type Inference for Python
Yun Peng†, Chaozheng Wang†, Wenxuan Wang†, Cuiyun Gao‡, Michael R. Lyu†
†Computer Science and Engineering Department, The Chinese University of Hong Kong, Hong Kong, China
‡School of Computer Science and Technology, Harbin Institute of Technology, Shenzhen, China
{'source': 'data/genAi_python_type_inference.pdf', 'page': 0, 'start_index': 0}
Saved 733 chunks to chromaDB.


/tmp/ipykernel_153330/237340530.py:28: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


## Query the VectorDB

In [32]:
query_text = "How Generative AI is used to infer python types?"

PROMPT_TEMPLATE = """
Answer the question based on the following context:
{context}
However this context might contain incomplete statements. So understand the content inherent context to provide insightful answers

====

Answer the question based on the above context: {question}
"""

In [33]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
    
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  
  load_dotenv()
  GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
  
  # YOU MUST - Use same embedding function as before
  embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

  # Prepare the database
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
  
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=3)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
  
  print("CONTEXT TEXT: \n", context_text)
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  
  # Initialize OpenAI chat model
  model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.5, max_tokens=1048, google_api_key=GOOGLE_API_KEY)

  # Generate response text based on the prompt
  response_text = model.invoke(prompt)  # model.predict() but its depricated
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"\nResponse: {response_text}\nSources: {sources}"
  return formatted_response, response_text

In [36]:

# Let's call our function we have defined
formatted_response, response_text = query_rag(query_text)
# and finally, inspect our final response!
print(response_text)

Unable to find matching results.
CONTEXT TEXT: 
 GPT-Neo 1.3B/2.7B The Pile Generative
InCoder 1.3B/6.7BGitHub &
Stack OverflowGenerative
& Infilling
CodeGen 6BThe Pile, BigQuery
& GitHubGenerative
GPT-J 6.7B The Pile Generative
GPT-3.5 175B - Generative
ChatGPT 175B - Generative
•Type4Py [27] is a supervised type inference approach. It

 - -

ChatGPT 175B - Generative
•Type4Py [27] is a supervised type inference approach. It
builds different type clusters and classifies a new Python
program into one of the type clusters to determine the
type.
•HiTyper [34] is a hybrid type inference approach. It

 - -

ative type inference approach for Python programs. T YPEGEN
has four phases, including code slicing, type hint collection,
chain-of-thought (COT) prompt construction, and type gen-
eration. In the code slicing phase, T YPEGENgenerates type
dependency graphs (TDGs) and builds code slices based on
content="The provided context highlights several approaches to type inference in Python, inc